# Continuous Control

---

Congratulations for completing the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control an agent in a more challenging environment, where the goal is to train a creature with four arms to walk forward.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Crawler.app"`
- **Windows** (x86): `"path/to/Crawler_Windows_x86/Crawler.exe"`
- **Windows** (x86_64): `"path/to/Crawler_Windows_x86_64/Crawler.exe"`
- **Linux** (x86): `"path/to/Crawler_Linux/Crawler.x86"`
- **Linux** (x86_64): `"path/to/Crawler_Linux/Crawler.x86_64"`
- **Linux** (x86, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86"`
- **Linux** (x86_64, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86_64"`

For instance, if you are using a Mac, then you downloaded `Crawler.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Crawler.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher_Windows_x86_64/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

### Simple Reinforce Algorithm

Here a simple reinforce algorithm is trained to solve the reacher problem. Even after 800 episodes there is only litte progress. The high variance of this monte carlo approach leads to a very slow rate of convergence.

In [ ]:
from pi_network import PiNetwork
from reinforce import reinforce
device='cpu'
policy = PiNetwork().to(device)


In [ ]:
policy, scores = reinforce(policy, env, brain_name, n_episodes=100)

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    actions,_ = policy.act(states)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                        # get reward (for each agent)
    print(rewards)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### Actor Critic

A deep deterministic policy gradient (DDPG) actor critic method is trained. For training a replay buffer of size 100000 with a batch size of 128 is used. The policy (actor) network consists of two hidden layers with 64 neurons with relu activation and tanh as activation in the output layer. The Q-network (critic) also consists of two hidden layers with 128 neurons both and relu activation.

In order to control the eploration of the agent an Ornstein-Uhlenbeck process is used. The parameters are as follows 
   - $\theta = 0.05$ (mean reversion rate)
   - $\mu = 0.0$ (mean)
   - $\sigma = 0.2$ (standard deviation)
    
Sigma decays over the number of episodes with a factor of $\sigma_{decay}=0.995$ until it reaches its minimum $\sigma_{min}=0.01$.

- $\gamma$: Discount factor during the computation of the target $(target = reward + gamma*Qvalue(state_{t+1}, action_{t+1})$.
- $\tau$, update_every: $\tau$ is a float value used to compute the convex combination of old Q-network's and target Q-network's weights. The weights are updated every update_every steps to stabilize the learning process.
- $lr$: Learning rates for the optimizer of the Q-network and the Policy network.


During each training step the Q-network is optimized by minimizing the mean squared error of the temporal difference $ reward + gamma*Qvalue(state_{t+1}, action_{t+1}) - Qvalue(state_t, action_t)$. Thus learning to estimate the value of a certain action in a certain state. The policy network is then optimized to choose actions with a high value via optimizing the policy network weights to maximize $Q(state, policy(state))$.
   
The replay buffer breaks the correlation between samples in a consecutive episode and therefore stabilizes the learning process.

In [4]:
BUFFER_SIZE = 100000  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 5e-4              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-4        # learning rate of the critic
WEIGHT_DECAY = 0.0001   # L2 weight decay

In [7]:
from pi_network import PiNetwork
from q_network import QNetwork

actor = PiNetwork(LR_ACTOR)
critic = QNetwork(input_size = state_size + action_size, lr = LR_CRITIC)

In [9]:
from agent import ActorCriticAgent
from ounoise import OUNoise
from replay_buffer import ReplayBuffer
from trainer import Trainer

batch_size = 128

memory = ReplayBuffer(state_size, BUFFER_SIZE , BATCH_SIZE, 1337)
noise = OUNoise(action_size, 1, 1337, sigma_decay=0.995, sigma_min=0.01)
ac = ActorCriticAgent(gamma=GAMMA, tau=TAU, batch_size=BATCH_SIZE, update_every=1,
                      actor=actor, critic=critic, memory=memory, ou_noise=noise)


In [10]:
trainer = Trainer(env, ac)

In [ ]:
scores = trainer.train(4000)

Episode 0	Average Score: 0.90,   0.90
Episode 100	Average Score: 2.26,   3.31
Episode 104	Average Score: 2.50,   3.54

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(1)                                   # initialize the score (for each agent)
while True:
    actions = trainer.agent.act(states)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
from agent import ActorCriticAgent
from ounoise import OUNoise
from replay_buffer import ReplayBuffer
from trainer import Trainer

batch_size = 128

memory = ReplayBuffer(33, 100000, batch_size , 1337)
noise = OUNoise(4, 1, 1337, sigma_decay=0.995, sigma_min=0.01)
ac = ActorCriticAgent(gamma=0.99, tau=5e-4, batch_size=batch_size, update_every=1, actor=actor, critic=critic, memory=memory, ou_noise=noise)


In [ ]:
trainer = Trainer(env, ac)

In [ ]:
scores = trainer.train(4000)

In [ ]:
np.zeros(shape=(20))

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(1)                          # initialize the score (for each agent)
while True:
    #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    actions = trainer.agent.act(states)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                        # get reward (for each agent)
    #print(rewards)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
dqn.train(150)

In [ ]:
pi_network = PiNetwork()

In [ ]:
state

In [ ]:
state = np.random.normal(size=(1,33))

In [ ]:
action, log_prob = pi_network.act(state)

In [ ]:
action

In [ ]:
log_prob

In [ ]:
sigma

In [ ]:
from torch.distributions.normal import Normal
m = Normal(mu, sigma)
actions = m.sample()

In [ ]:
actions

In [ ]:
m.log_prob(actions)

In [ ]:
actions

In [ ]:
state

In [ ]:
state = np.random.normal(size=(1,33))
state = torch.from_numpy(state).float().unsqueeze(0).to(device)
mu, sigma = pi_network.forward(state)
mu = mu.cpu()
sigma = sigma.cpu()

m = Normal(mu, sigma)
actions = m.sample()
log_probs = m.log_prob(actions[0])
actions = actions.cpu().data.numpy()
#np.clip(actions, -1, 1), log_probs.prod().reshape(-1)

In [ ]:
log_probs.prod().reshape(-1)

In [ ]:
m.sample()

In [ ]:
actions

In [ ]:
log_probs

In [ ]:
log_probs.prod().reshape(-1)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.distributions as dist
x = torch.tensor([1.0,2,3,4])
y = torch.tensor([1.0,2,3,4])

k = nn.Parameter(0.01*torch.randn(1))

d = dist.Normal(loc=k*x, scale=1)

log_prob = d.log_prob(y)

In [ ]:
log_prob

In [ ]:
np.prod([1,2,3])

In [ ]:
import numpy as np
import torch
from torch import optim
import torch.nn as nn
from collections import OrderedDict
from torch.distributions.normal import Normal
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'


class PiNetwork(nn.Module):
    def __init__(self, lr):
        super(PiNetwork, self).__init__()

        self.model_mu_sigma = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(33, 256)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(256, 128)),
            ('relu2', nn.ReLU()),
            ('logits', nn.Linear(128, 4*2)),
            ('tanh', nn.Tanh())
        ]))
        self.optimizer = optim.Adam(self.model_mu_sigma.parameters(), lr=lr)

    def forward(self, state):
        return self.model_mu_sigma.forward(state)#.reshape(1, -1)

    def act(self, state):
        state = torch.from_numpy(state).float().to(device)
        mu_sigma = self.forward(state)
        m = Normal(mu_sigma[:,:4], torch.clamp(0.5+0.5*mu_sigma[:,4:], min=0.05))
        actions = m.sample()
        log_probs = m.log_prob(actions)
        actions = actions.cpu().data.numpy().reshape(-1, 4)
        return np.clip(actions, -1, 1), log_probs.sum(axis=1)

    def clone(self):
        qnetwork_clone = PiNetwork()
        qnetwork_clone.model_mu_sigma = copy.deepcopy(self.model_mu_sigma)
        return qnetwork_clone

In [ ]:
pi = PiNetwork(0.001)

In [ ]:
pi.forward(torch.Tensor(np.random.normal(size=(20,33)))).shape

In [ ]:
a,prob = pi.act(np.random.normal(size=(20,33)))

In [ ]:
a.shape

In [ ]:
a

In [ ]:
prob.shape

In [ ]:
prob.shape

In [ ]:
 m = Normal(torch.Tensor([[]]), 0.00001*torch.clamp(0.5+0.5*mu_sigma[:,4:], min=0.05))